In [2]:
print("hello")

hello


In [33]:
%pwd

'd:\\DATA SCIENCE\\ML project\\Airline_customer_statisfaction_prediction_with_MLOPS\\research'

In [34]:
import os
os.chdir("../")

In [35]:
%pwd

'd:\\DATA SCIENCE\\ML project\\Airline_customer_statisfaction_prediction_with_MLOPS'

In [48]:

import mysql.connector
from mysql.connector import Error
import os
import csv
import sys
from pathlib import Path
from dataclasses import dataclass
from contextlib import contextmanager

from src.airline.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.airline.utils.comman import read_yaml_file, create_directories
from src.airline.exception import customException
from src.airline.logger import logging

In [49]:
@dataclass(frozen=True)
class DataIngestionConfig:
    host: str
    user: str
    password: str
    database: str
    table_name: str
    root: Path
    local_data_file: Path


In [55]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH):
        self.config = read_yaml_file(config_filepath)

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config["data_ingestion"]
        create_directories([config["root_dir"]])

        return DataIngestionConfig(
            root=Path(config["root_dir"]),
            host=config["host"],
            user=config["user"],
            password=config["password"],
            database=config["database"],
            table_name=config["table_name"],
            local_data_file=Path(config["local_data_file"]),
        )

In [56]:


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.connection = None

    @contextmanager
    def get_connection(self):
        """Context manager for MySQL connection."""
        connection = mysql.connector.connect(
            host=self.config.host,
            user=self.config.user,
            password=self.config.password,
            database=self.config.database
        )
        try:
            yield connection
        finally:
            if connection.is_connected():
                connection.close()
                logging.info("Database connection closed successfully.")


In [57]:
class MySqlDataExtractor:
    def __init__(self, config: DataIngestionConfig):
        self.db = DataIngestion(config)
        self.table_name = config.table_name

    def extract_to_csv(self, output_dir: Path):
        """Extract data from MySQL table and save to a CSV file."""
        try:
            with self.db.get_connection() as connection:
                cursor = connection.cursor()
                query = f"SELECT * FROM {self.table_name}"
                cursor.execute(query)

                rows = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]

                logging.info("Data fetched successfully.")

                # Ensure output directory exists
                output_dir.mkdir(parents=True, exist_ok=True)
                csv_file = output_dir / f"{self.table_name}_data.csv"

                with csv_file.open(mode="w", newline="", encoding="utf-8") as file:
                    writer = csv.writer(file)
                    writer.writerow(columns)
                    writer.writerows(rows)

                logging.info(f"Data successfully saved to {csv_file}")

        except Exception as e:
            logging.error(f"Error while extracting data from the database: {str(e)}")
            raise customException(e, sys)

In [58]:
try:
     config_manager = ConfigurationManager()
     data_ingestion_config = config_manager.get_data_ingestion_config()

        # Initialize Data Extraction and Extraction Process
     data_extractor = MySqlDataExtractor(config=data_ingestion_config)

        # Extract data and save to CSV
     data_extractor.extract_to_csv(output_dir=data_ingestion_config.root / "extracted_data")

except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise e

: 

In [1]:
# Check if the connection works outside the script
import mysql.connector

try:
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="password@123",
        database="airline_database"
    )
    print("Connection successful!")
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if connection and connection.is_connected():
        connection.close()
        print("Connection closed.")


: 